<a href="https://colab.research.google.com/github/Jasminewakini/WK-5-Electric-Car-Sharing-Project/blob/main/electric_car_sharing_python_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overview

I will be working as a data scientist working for an electric car-sharing service company. I have been tasked to process stations data to understand electric car usage over time by solving for the following research question;

Research Question;

1. Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.


2. What is the most popular hour for returning cars?

3. What station is the most popular?

    a) Overall?

    b) At the most popular picking hour?
  

4. What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?

    a) Overall?

    b) At the most popular picking hour?

5. Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 






# 1.0 Importing our Libraries

In [1]:
# Importing the libraries we will need for this project

# Importing the pandas library

import pandas as pd

# Importing the numpy library

import numpy as np

# importing the datetime library

from datetime import datetime

# 1.1 Reading the Dataset from our CSV file

Dataset link for download or access: [Link](https://bit.ly/autolib_dataset)

In [2]:
# We will first import our dataset that we will use for the examples. We will do this by first
# then storing the url location of our dataset to the variable url
url = 'http://bit.ly/autolib_dataset'

# We will read the dataset from above url and store the dataframe in the variable df
autolib = pd.read_csv(url)
autolib

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27


# 1.2 Previewing our Dataset

In [3]:
# Let's preview the first 10 rows of our data
# 
autolib.head(10)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
5,8 Boulevard Voltaire,0,0,0,0,0,nonexistent,Paris,NaN,paris-voltaire-8,STATION,"48.8657658, 2.3664376",75011,Paris/Voltaire/8,operational,NaN,4,station,ok,nonexistent,2018,4,6,7,2
6,37 rue Leblanc,0,0,0,0,0,nonexistent,Paris,"Station en parking (niv -1), accÃ¨s 37 rue Leb...",paris-citroencevennes-parking,PARKING,"48.83874, 2.27641",75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,NaN,0,station,closed,nonexistent,2018,4,8,18,20
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,0,nonexistent,Nanterre,NaN,nanterre-luaps-17,STATION,"48.88069, 2.21063",92000,Nanterre/Luaps/17,operational,NaN,0,station,ok,nonexistent,2018,4,4,22,13
8,34 avenue Jean Moulin,1,1,0,0,0,nonexistent,Paris,NaN,paris-jeanmoulin-34,STATION,"48.8266807, 2.3237355",75014,Paris/Jean Moulin/34,operational,NaN,4,station,ok,nonexistent,2018,4,2,22,58
9,41 boulevard de Rochechouart,6,6,0,0,0,nonexistent,Paris,NaN,paris-anvers-parking,PARKING,"48.88267, 2.34405",75009,Paris/Anvers/Parking,operational,NaN,0,station,ok,nonexistent,2018,4,4,15,2


# 1.3 Accessing Information about our Dataset

In [4]:
# Getting to know more about the dataset by accessing its information
# 
autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

# 1.4 Cleaning our Dataset

Let us perform data cleaning procedures below providing a documentation for our actions and reasons

### 1.41) Validity

In [5]:
# Procedure 1: Irrelevant Data Observation
# Data Cleaning Action: Dropping columns
# Explanation: No question to be answered require the columns dropped columns.
#
autolib.drop(['Displayed comment', 'Geo point', 'Charge Slots', 'Charging Status', 'Scheduled at', 'Station type' , 'Subscription status'], axis = 1, inplace = True)
autolib

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,ID,Kind,Postal code,Public name,Rental status,Slots,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,2,0,0,Colombes,colombes-menelotte-8,STATION,92700,Colombes/MÃ©nelotte/8,operational,3,ok,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,Paris,paris-dantzig-37,STATION,75015,Paris/Dantzig/37,operational,2,ok,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,Paris,paris-bac-142,STATION,75007,Paris/Bac/142,operational,4,ok,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,Fontenay-Sous-Bois,fontenaysousbois-valdefontenay-2,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,3,ok,2018,4,4,17,27


In [6]:
# Procedure 2: Outliers
# Data Cleaning Action: Identifying the outliers
# Explanation: we identify the ouliers and decide whether to keep or drop them  .
# Calculating our first, third quantiles and then later our IQR
#

Q1 = autolib.quantile(0.25)
Q3 = autolib.quantile(0.75)
IQR = Q3 - Q1
IQR

# The rule of thumb is that anything not in the range of 
# (Q1 - 1.5 IQR) and (Q3 + 1.5 IQR) is an outlier, and can be removed.
#

(autolib < (Q1 - 1.5 * IQR)) |(autolib > (Q3 + 1.5 * IQR))

# Uncommenting the line of code below is how we removing outliers based on the IQR range and stores the result in the data frame 'electric_out'
# but for now we are going to keep them

# autolib_out = autolib[~((autolib < (Q1 - 1.5 * IQR)) |(autolib > (Q3 + 1.5 * IQR))).any(axis=1)]
# autolib_out

,Address,Bluecar counter,Cars,City,ID,Kind,Postal code,Public name,Rental status,Slots,Status,Utilib 1.4 counter,Utilib counter,day,hour,minute,month,year
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4996,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4997,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4998,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


### 1.42) Accuracy

In [12]:
# Procedure 1: In-record & cross-datasets errors
# Data Cleaning Action: No action taken
# Explanation: the dataset was accurate no fixing needed.

### 1.43) Completeness

In [7]:
# Procedure 1: missing values
# Data Cleaning Action: checking for missing values
# Explanation: we need to check if we have missing values 
#
autolib.isnull().sum()

# no missing values

Address               0
Cars                  0
Bluecar counter       0
Utilib counter        0
Utilib 1.4 counter    0
City                  0
ID                    0
Kind                  0
Postal code           0
Public name           0
Rental status         0
Slots                 0
Status                0
year                  0
month                 0
day                   0
hour                  0
minute                0
dtype: int64

### 1.44) Consitency

In [8]:
# Procedure 1: duplicates
# Data Cleaning Action: checking for dupicates in our dataset
# Explanation: Duplicates are data points that are repeated in your dataset. 
# These should be simply removed.
#
autolib[autolib.duplicated()]

# Identifying which observations are duplicates
autolib2 = autolib.drop_duplicates()

# previwing for the non duplicates

autolib2.head()


,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,ID,Kind,Postal code,Public name,Rental status,Slots,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23


### 1.45) Uniformity

In [9]:
# Procedure 1: Standardization
# Data Cleaning Action: Fixing messy column names
# Explanation: We have column names which are in uppercase, with spaces, 
# and whitespace all around we remove this to make the column names standard
#

autolib2.columns = autolib2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
autolib2.head()

,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23


In [11]:
# Procedure 1:  Date Type Conversion
# Data Cleaning Action: Combining the columns year,day and time into one column and changing the column to datatime datatype
# Explanation: We have change the columns year,month,day and time to there respective data type


autolib2['DateTime']=pd.to_datetime(autolib2[['year','month','day', 'hour', 'minute']])
autolib2.head()

,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute,DateTime
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43,2018-04-08 11:43:00
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24,2018-04-06 07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14,2018-04-03 20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37,2018-04-04 04:37:00
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23,2018-04-08 17:23:00


In [12]:
autolib2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   address             5000 non-null   object        
 1   cars                5000 non-null   int64         
 2   bluecar_counter     5000 non-null   int64         
 3   utilib_counter      5000 non-null   int64         
 4   utilib_1.4_counter  5000 non-null   int64         
 5   city                5000 non-null   object        
 6   id                  5000 non-null   object        
 7   kind                5000 non-null   object        
 8   postal_code         5000 non-null   int64         
 9   public_name         5000 non-null   object        
 10  rental_status       5000 non-null   object        
 11  slots               5000 non-null   int64         
 12  status              5000 non-null   object        
 13  year                5000 non-null   int64       

# 1.5 Answering Research Questions

### Bluecar Counter

In [13]:
# To compute usage, we will need to understand that we have to join successive (in time) measures/counters for a given station, 
# as the difference will tell whether a car was picked up, returned, or nothing happened.
# We create a new column where we find this difference in the Bluecar Counter
#

autolib2['bluecar_diff'] = autolib2['bluecar_counter'].diff()
autolib2.head(10)

,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute,DateTime,bluecar_diff
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43,2018-04-08 11:43:00,NaN
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24,2018-04-06 07:24:00,6.0
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14,2018-04-03 20:14:00,-3.0
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37,2018-04-04 04:37:00,0.0
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23,2018-04-08 17:23:00,0.0
5,8 Boulevard Voltaire,0,0,0,0,Paris,paris-voltaire-8,STATION,75011,Paris/Voltaire/8,operational,4,ok,2018,4,6,7,2,2018-04-06 07:02:00,-3.0
6,37 rue Leblanc,0,0,0,0,Paris,paris-citroencevennes-parking,PARKING,75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,0,closed,2018,4,8,18,20,2018-04-08 18:20:00,0.0
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,Nanterre,nanterre-luaps-17,STATION,92000,Nanterre/Luaps/17,operational,0,ok,2018,4,4,22,13,2018-04-04 22:13:00,3.0
8,34 avenue Jean Moulin,1,1,0,0,Paris,paris-jeanmoulin-34,STATION,75014,Paris/Jean Moulin/34,operational,4,ok,2018,4,2,22,58,2018-04-02 22:58:00,-2.0
9,41 boulevard de Rochechouart,6,6,0,0,Paris,paris-anvers-parking,PARKING,75009,Paris/Anvers/Parking,operational,0,ok,2018,4,4,15,2,2018-04-04 15:02:00,5.0


In [14]:
# Question 1
# Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018

# We specify that city == Paris

# positive value mean a car was returned

# Negative value means a car was taken

# Zero shows that there was no change

bluecar_paris = autolib2[autolib2['city'] =='Paris'] 

bluecar_paris.head()

,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute,DateTime,bluecar_diff
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43,2018-04-08 11:43:00,NaN
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24,2018-04-06 07:24:00,6.0
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37,2018-04-04 04:37:00,0.0
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23,2018-04-08 17:23:00,0.0
5,8 Boulevard Voltaire,0,0,0,0,Paris,paris-voltaire-8,STATION,75011,Paris/Voltaire/8,operational,4,ok,2018,4,6,7,2,2018-04-06 07:02:00,-3.0


In [15]:
# To identify the most popular hour of the day for picking up 
#
bluecar_paris[bluecar_paris['bluecar_diff'] < 0].groupby('hour')['hour'].count().sort_values(ascending= False)

hour
21    80
16    68
7     64
13    62
5     62
23    61
20    61
3     60
8     59
4     58
18    57
9     57
17    55
19    55
22    55
14    54
12    54
0     52
10    52
2     51
6     50
15    49
11    47
1     43
Name: hour, dtype: int64

The most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris is 2100hr

In [16]:
# Question 2
# What is the most popular hour for returning cars?
#
bluecar_paris[bluecar_paris['bluecar_diff'] > 0].groupby('hour')['hour'].count().sort_values(ascending= False)

hour
3     49
12    48
10    48
6     46
8     44
21    43
11    42
19    42
15    42
18    40
9     40
0     39
22    38
2     38
1     38
16    37
5     37
23    35
13    33
7     33
17    33
4     31
20    27
14    25
Name: hour, dtype: int64

The most popular hour of the day for returning a shared electric car (Bluecar) in the city of Paris is 0300hr

In [17]:
# Question 3a:
# What station is the most popular overall?
# 

bp = bluecar_paris[(bluecar_paris['kind'] == 'STATION') & (bluecar_paris['status'] == 'ok')]
bp.groupby('public_name')['public_name'].count().sort_values(ascending= False)

public_name
Paris/Porte de Montrouge/8         13
Paris/Voltaire/182                 11
Paris/Courcelles/69                11
Paris/Mathis/35                    11
Paris/Philippe Auguste/126         11
                                   ..
Paris/PÃ©guy/2                      1
Paris/Faubourg-Saint-Martin/168     1
Paris/Bobillot/16                   1
Paris/FÃ©dÃ©ration/10               1
Paris/Murat/157                     1
Name: public_name, Length: 518, dtype: int64

The station that is the most popular overall is Paris/Porte de Montrouge/8

In [18]:
# Question 3b:
# What station is the most popular At the most popular picking hour?
# 

bp = bluecar_paris[(bluecar_paris['kind'] == 'STATION') & (bluecar_paris['status'] == 'ok') & (bluecar_paris['hour'] == 21)]
bp.groupby('public_name')['public_name'].count().sort_values(ascending= False)

public_name
Paris/Tronchet/19               4
Paris/Voltaire/182              3
Paris/Ãmile Reynaud/4          2
Paris/Danton/8                  2
Paris/Boulard/1                 2
                               ..
Paris/Pelleport/86              1
Paris/Pereire/170               1
Paris/Pompe/137                 1
Paris/Pont Louis-Philippe/24    1
Paris/25 AoÃ»t 1944/8           1
Name: public_name, Length: 106, dtype: int64

The station that is the most popular at the most popular picking hour ie 2100hr is Paris/Tronchet/19 

In [19]:
# Question 4a:
# What postal code is the most popular for picking up Blue cars Overall?
# 

bp = bluecar_paris[(bluecar_paris['kind'] == 'STATION') & (bluecar_paris['status'] == 'ok')]
bp.groupby('postal_code')['postal_code'].count().sort_values(ascending= False)

# Does the most popular station belong to that postal code?
# The postal code with 75015 is Paris/Suffren/2 and the most popular station is Paris/Porte de Montrouge/8 
# therfore the most popular station does not belong to that postal code

postal_code
75015    220
75016    197
75017    180
75011    155
75013    155
75008    150
75020    149
75012    138
75019    134
75014    128
75018    120
75007     99
75010     98
75009     93
75005     85
75006     74
75004     58
75003     55
75002     54
75001     43
75112      6
75116      5
Name: postal_code, dtype: int64

The postal code that is the most popular for picking up Blue cars Overall 75015

In [20]:
# Question 4b:
# What postal code is the most popular for picking up Blue cars at the most popular picking hour?
# 

bp = bluecar_paris[(bluecar_paris['kind'] == 'STATION') & (bluecar_paris['status'] == 'ok') & (bluecar_paris['hour'] == 21)]
bp.groupby('postal_code')['postal_code'].count().sort_values(ascending= False)

# Does the most popular station belong to that postal code?
# The postal code with 75014 is Paris/Raymond Losserand/145 and the most popular station at the most popular picking hour 
# is Paris/Tronchet/19  therfore the most popular station does not belong to that postal code

postal_code
75014    15
75019    12
75012    11
75008    11
75016    10
75013     9
75011     8
75015     7
75020     5
75017     5
75007     5
75009     4
75006     3
75003     3
75010     3
75018     3
75005     2
75004     2
75001     2
75002     1
Name: postal_code, dtype: int64

The postal code that is the most popular for picking up Blue cars at the most popular picking hour ie 2100hr is 75014

### Utilib Counter

In [21]:
# We create a new column where we find this difference in the Bluecar Counter
#

autolib2['utilib_diff'] = autolib2['utilib_counter'].diff()
autolib2.head(10)

,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute,DateTime,bluecar_diff,utilib_diff
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43,2018-04-08 11:43:00,NaN,NaN
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24,2018-04-06 07:24:00,6.0,0.0
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14,2018-04-03 20:14:00,-3.0,0.0
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37,2018-04-04 04:37:00,0.0,1.0
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23,2018-04-08 17:23:00,0.0,-1.0
5,8 Boulevard Voltaire,0,0,0,0,Paris,paris-voltaire-8,STATION,75011,Paris/Voltaire/8,operational,4,ok,2018,4,6,7,2,2018-04-06 07:02:00,-3.0,0.0
6,37 rue Leblanc,0,0,0,0,Paris,paris-citroencevennes-parking,PARKING,75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,0,closed,2018,4,8,18,20,2018-04-08 18:20:00,0.0,0.0
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,Nanterre,nanterre-luaps-17,STATION,92000,Nanterre/Luaps/17,operational,0,ok,2018,4,4,22,13,2018-04-04 22:13:00,3.0,1.0
8,34 avenue Jean Moulin,1,1,0,0,Paris,paris-jeanmoulin-34,STATION,75014,Paris/Jean Moulin/34,operational,4,ok,2018,4,2,22,58,2018-04-02 22:58:00,-2.0,-1.0
9,41 boulevard de Rochechouart,6,6,0,0,Paris,paris-anvers-parking,PARKING,75009,Paris/Anvers/Parking,operational,0,ok,2018,4,4,15,2,2018-04-04 15:02:00,5.0,0.0


In [22]:
# Question 1
# Identify the most popular hour of the day for picking up a shared electric car (Utilib) in the city of Paris over the month of April 2018
#
Utilib_paris = autolib2[autolib2['city'] =='Paris'] 

Utilib_paris.head()
Utilib_paris[Utilib_paris['utilib_diff'] < 0].groupby('hour')['hour'].count().sort_values(ascending= False)


hour
19    12
23    10
22    10
14    10
6      9
10     9
5      8
8      8
9      8
3      7
11     7
15     6
16     6
17     6
7      6
20     6
2      6
13     5
4      5
0      5
12     4
1      4
18     3
21     3
Name: hour, dtype: int64

The most popular hour of the day for picking up a shared electric car (Utilib) in the city of Paris over the month of April 2018 is 1900hr

In [23]:
# Question 2
# What is the most popular hour for returning cars?
#
Utilib_paris[Utilib_paris['utilib_diff'] > 0].groupby('hour')['hour'].count().sort_values(ascending= False)

hour
6     12
11     9
17     9
8      9
14     8
10     7
21     7
18     7
7      7
15     6
2      5
3      5
5      5
0      5
12     5
13     5
9      4
16     4
22     3
4      3
19     3
20     3
1      3
23     1
Name: hour, dtype: int64

The most popular hour for returning these cars is 0600hr

In [24]:
# Question 3a:
# What station is the most popular overall?
# 

Utilib_paris.groupby('public_name')['utilib_diff'].count().sort_values(ascending=False)


public_name
Paris/Porte de Montrouge/8    13
Paris/Mathis/35               11
Paris/Courcelles/69           11
Paris/Voltaire/182            11
Paris/Grenelle 31/Parking     11
                              ..
Paris/Pirogues de Bercy/44     1
Paris/Porte Pouchet/6          1
Paris/FÃ©dÃ©ration/10          1
Paris/Lecourbe/336             1
Paris/Pyramides/Parking        1
Name: utilib_diff, Length: 605, dtype: int64

The station is the most popular overall is Paris/Porte de Montrouge/8

In [25]:
# Question 3b:
# What station is the most popular At the most popular picking hour?
# 
Utilib_paris[(Utilib_paris.hour == 19)]['public_name'].value_counts()

Paris/Perle/18                2
Paris/Clichy/181              2
Paris/Charenton/251           2
Paris/Porte de Montrouge/8    2
Paris/Pierre Chausson/1       2
                             ..
Paris/Raspail/136             1
Paris/Caulaincourt/71         1
Paris/Commandant Lamy/2       1
Paris/SÃ©gur/55               1
Paris/Parmentier/62           1
Name: public_name, Length: 105, dtype: int64

The station that is the most popular at the most popular picking hour ie 1900hr is Paris/Perle/18 

In [26]:
# Question 4a:
# What postal code is the most popular for picking up Utilib cars Overall?
# 


uc = Utilib_paris[(Utilib_paris.city == 'Paris') & (Utilib_paris.utilib_diff <= 0)]
uc.groupby('postal_code')['hour'].count().sort_values(ascending=False)

postal_code
75015    257
75016    216
75017    203
75011    175
75008    165
75012    164
75013    156
75020    156
75019    153
75014    143
75018    135
75010    112
75007    107
75009    102
75006     92
75005     90
75003     66
75004     66
75002     59
75001     52
75112      6
75116      5
Name: hour, dtype: int64

The postal code that is the most popular for picking up Utilib cars Overall is 75015

In [27]:
# Question 4b:
# What postal code is the most popular for picking up Utilib cars at the most popular picking hour?
# 
uc = Utilib_paris[(Utilib_paris['kind'] == 'STATION') & (Utilib_paris['status'] == 'ok') & (Utilib_paris['hour'] == 19)] 
uc.groupby('postal_code')['postal_code'].count().sort_values(ascending= False)

postal_code
75020    9
75012    9
75016    8
75013    8
75011    8
75017    8
75015    6
75010    6
75003    6
75007    5
75014    4
75018    4
75019    4
75002    3
75008    2
75006    2
75004    2
75005    1
75001    1
Name: postal_code, dtype: int64

The postal code that is the most popular for picking up Utilib cars at the most popular picking hour ie 1900hr is 75020

### Utilib 1.4 Counter

In [28]:
# Creating a new column where we find the difference in the number of Utilib 1.4 counter


#bluecar_paris.drop(["utilib_two_diff", "utilib_1.4_diff"], axis = 1, inplace = True) 

autolib2['utilib_two_diff'] = autolib2['utilib_1.4_counter'].diff()
autolib2.head(10)
# bluecar_paris['utilib_two_diff'] = bluecar_paris['utilib_1.4_counter'].diff()


,address,cars,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,slots,status,year,month,day,hour,minute,DateTime,bluecar_diff,utilib_diff,utilib_two_diff
0,2 Avenue de Suffren,0,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,2,ok,2018,4,8,11,43,2018-04-08 11:43:00,NaN,NaN,NaN
1,145 Rue Raymond Losserand,6,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,0,ok,2018,4,6,7,24,2018-04-06 07:24:00,6.0,0.0,0.0
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,ok,2018,4,3,20,14,2018-04-03 20:14:00,-3.0,0.0,2.0
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,ok,2018,4,4,4,37,2018-04-04 04:37:00,0.0,1.0,-2.0
4,6 avenue de la Porte de Champerret,3,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,3,ok,2018,4,8,17,23,2018-04-08 17:23:00,0.0,-1.0,0.0
5,8 Boulevard Voltaire,0,0,0,0,Paris,paris-voltaire-8,STATION,75011,Paris/Voltaire/8,operational,4,ok,2018,4,6,7,2,2018-04-06 07:02:00,-3.0,0.0,0.0
6,37 rue Leblanc,0,0,0,0,Paris,paris-citroencevennes-parking,PARKING,75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,0,closed,2018,4,8,18,20,2018-04-08 18:20:00,0.0,0.0,0.0
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,Nanterre,nanterre-luaps-17,STATION,92000,Nanterre/Luaps/17,operational,0,ok,2018,4,4,22,13,2018-04-04 22:13:00,3.0,1.0,0.0
8,34 avenue Jean Moulin,1,1,0,0,Paris,paris-jeanmoulin-34,STATION,75014,Paris/Jean Moulin/34,operational,4,ok,2018,4,2,22,58,2018-04-02 22:58:00,-2.0,-1.0,0.0
9,41 boulevard de Rochechouart,6,6,0,0,Paris,paris-anvers-parking,PARKING,75009,Paris/Anvers/Parking,operational,0,ok,2018,4,4,15,2,2018-04-04 15:02:00,5.0,0.0,0.0


In [29]:
# Question 1
# Identify the most popular hour of the day for picking up a 
# shared electric car (Utilib 1.4 counter) in the city of Paris over the month of April 2018
#
Utilib2_paris = autolib2[autolib2['city'] =='Paris'] 

Utilib2_paris.head()
Utilib2_paris[Utilib2_paris['utilib_two_diff'] < 0].groupby('hour')['hour'].count().sort_values(ascending= False)

hour
3     20
4     17
0     16
19    16
8     16
10    16
12    15
14    12
2     12
23    12
16    12
17    12
20    12
21    12
11    12
22    11
7     11
18    11
9     10
5     10
13     8
6      8
15     7
1      7
Name: hour, dtype: int64

The most popular hour of the day for picking up a shared electric car (Utilib 1.4) in the city of Paris over the month of April 2018 is 0300hr

In [30]:
# Question 2
# What is the most popular hour for returning cars?
#
Utilib2_paris[Utilib2_paris['utilib_two_diff'] > 0].groupby('hour')['hour'].count().sort_values(ascending= False)

hour
9     17
17    15
5     15
21    14
22    14
10    13
6     12
15    12
3     12
19    11
16    11
1     11
12    11
20    10
18    10
23     9
11     9
7      9
13     9
8      8
4      8
14     8
0      7
2      6
Name: hour, dtype: int64

The most popular hour for returning cars is 0900hr

In [31]:
# Question 3a:
# What station is the most popular overall?
# 

uc2 = Utilib2_paris[(Utilib2_paris['kind'] == 'STATION') & (Utilib2_paris['status'] == 'ok')]
uc2.groupby('public_name')['utilib_two_diff'].count().sort_values(ascending=False)

public_name
Paris/Porte de Montrouge/8         13
Paris/Voltaire/182                 11
Paris/Courcelles/69                11
Paris/Mathis/35                    11
Paris/Philippe Auguste/126         11
                                   ..
Paris/PÃ©guy/2                      1
Paris/Faubourg-Saint-Martin/168     1
Paris/Bobillot/16                   1
Paris/FÃ©dÃ©ration/10               1
Paris/Murat/157                     1
Name: utilib_two_diff, Length: 518, dtype: int64

The station that is the most popular overall is Paris/Porte de Montrouge/8

In [32]:
# Question 3b:
# What station is the most popular At the most popular picking hour?
# 
uc2 = Utilib2_paris[(Utilib2_paris['kind'] == 'STATION') & (Utilib2_paris['status'] == 'ok')  & (Utilib2_paris['hour'] == 3)]
uc2.groupby('public_name')['utilib_two_diff'].count().sort_values(ascending=False)

public_name
Paris/Mortier/15             2
Paris/Julien Lacroix/39      2
Paris/Suffren/161            2
Paris/Quatre septembre/10    2
Paris/Francois 1er/7         2
                            ..
Paris/Pereire/115            1
Paris/PerrÃ©e/18             1
Paris/Plantes/56             1
Paris/PoissonniÃ¨re/64       1
Paris/25 AoÃ»t 1944/8        1
Name: utilib_two_diff, Length: 89, dtype: int64

The station that is the most popular at the most popular picking hour ie 0300hr is Paris/Mortier/15

In [33]:
# Question 4a:
# What postal code is the most popular for picking up Utilib cars Overall?
# 

uc = Utilib2_paris[(Utilib2_paris.city == 'Paris') & (Utilib2_paris.utilib_two_diff <= 0)]
uc.groupby('postal_code')['utilib_two_diff'].count().sort_values(ascending=False)

postal_code
75015    246
75016    217
75017    188
75011    170
75008    155
75013    155
75012    152
75020    149
75014    135
75019    130
75018    122
75010    107
75007    106
75009    101
75006     91
75005     86
75004     64
75003     62
75002     57
75001     51
75112      5
75116      5
Name: utilib_two_diff, dtype: int64

The postal code that is the most popular for picking up Utilib cars Overall is 75015

In [34]:
# Question 4b:
# What postal code is the most popular for picking up Utilib cars at the most popular picking hour?
# 
uc = Utilib2_paris[(Utilib2_paris['kind'] == 'STATION') & (Utilib2_paris['status'] == 'ok') & (Utilib2_paris['hour'] == 3)] 
uc.groupby('postal_code')['utilib_two_diff'].count().sort_values(ascending=False)

postal_code
75016    9
75015    8
75020    8
75012    7
75018    7
75017    7
75011    6
75013    5
75008    5
75006    5
75014    4
75009    4
75007    4
75003    4
75019    3
75004    3
75002    2
75010    2
75116    1
75001    1
Name: utilib_two_diff, dtype: int64

The postal code that is the most popular for picking up Utilib cars at the most popular picking hour ie 0300hr is 75016